# First model for warehouse picking optimisation

This notebook contains code to optimise the following problem.

Suppose we have a warehouse with $n$ unique products, and $n$ pick faces ($n$ must be even). The pick faces are laid out as below, in a single aisle on shelves with no shelves on top of each other, with odd number increasing on the left, and even numbers increasing from bottom up on the right. The diagram is viewed from the air, looking down, and let's say up is "north".

```
+---+    >.....v    +---+
|n-1|    :     :    | n |
+---+    ^.>...v    +---+
|   |    :     :    |   |

           ...

|   |    :     :    |   |
+---+    ^.>...v    +---+
| 7 |    :     :    | 8 |
+---+    ^.>...v    +---+
| 5 |    :     :    | 6 |
+---+    ^.>...v    +---+
| 3 |    :     :    | 4 |
+---+    ^.>...v    +---+
| 1 |    :     :    | 2 |
+---+    .     v    +---+
     @...^     :
 PC  #.<       >.. Sorting
       :              v
       ^..............<
```

The picking procedure (a single "pick wave") works as follows:

1. The picker starts at the PC, marked with an at sign (`@`) on the map
2. The picker prints out a flow path from the WMS+optimiser at the computer (`PC`)
3. The picker then starts walking up on the left hand side of the aisle, next to the odd pick faces.
4. If the picker has nothing to pick north of the current location, they can move to the right and start picking products in even pick faces, and walk south along the right side of the aisle.
5. Once the picker has picked all items from their shelves, they move down to the sorting facility (`Sorting` on the diagram)
6. The picker sorts and packs all orders into their correct shipping packages, and drops them off at the same location
7. The picker moves back to the PC and finishes at the hash sign (`#`) on the map

## The cost structure

We model the time taken to pick an order as the cost for this problem.

It's relatively difficult to estimate the actual time taken to complete each step of a pick wave, so we leave this in a highly modular and customisable system.

We basically assume the cost of a pick wave is a linear function of the following steps which may have more difficult, non-linear cost functions:

* $c_{\text{fixed}}$: A fixed cost of one pick wave (e.g. printing, moving from computer to first shelf, last shelf to sorting, sorting to computer, rest)
* $c_{\text{move}}$: Moving one shelf up or down the aisle (note that there are always an even number of these steps)
* $c_{\text{across}}$: Moving across the aisle from left to right
* $c_{\text{picking}}$: Picking objects from a shelf into containers, this is encoded as $p_1$, $p_2$, $\dots$, $p_q$ where $q$ is the number of containers, and $p_i$ is the number of identical objects picked form a shelf into container $i$. The cost for this tuple is a function $c_{\text{picking}}(q, (p_1, p_2, \dots, p_q))$.
* $c_{\text{sorting}}$: Sorting objects in $q$ containers to $r$ shipping "packages" (note the terminology: containers are used for picking from shelves, packages for final postable boxes), this is encoded as $g_{ij}$ with $i=1,\dots,q$ and $j=1,\dots,r$ for the number of items from container $i$ being sorted into package $j$, so the cost function is $c_{\text{sorting}}(q, r, ((g_{11}, g_{12}, \dots, g_{1r}), (g_{21}, g_{22}, \dots, g_{2r}), \dots, (g_{q1}, g_{q2}, \dots, g_{qr})))$

The total cost, $C:=c_{\text{total}}$, is then given by the sum of these costs:

$$
C=n_{\text{fixed}}\times c_{\text{fixed}}+n_{\text{move}}\times c_{\text{move}}+n_{\text{across}}\times c_{\text{across}}+\sum_{q,p_1,p_2,\dots,p_q}n_{\text{picking}}(q, (p_1, p_2, \dots, p_q))\times c_{\text{picking}}(q, (p_1, p_2, \dots, p_q))+\sum_{q,r,g_1,g_2,\dots,g_q,g_{11},\dots,g_{qr}}n_{\text{sorting}}(q, r, ((g_{11}, g_{12}, \dots, g_{1r}), (g_{21}, g_{22}, \dots, g_{2r}), \dots, (g_{q1}, g_{q2}, \dots, g_{qr})))c_{\text{sorting}}(q, r, ((g_{11}, g_{12}, \dots, g_{1r}), (g_{21}, g_{22}, \dots, g_{2r}), \dots, (g_{q1}, g_{q2}, \dots, g_{qr})))
$$

Where the $n_{\text{fixed}}$, $n_{\text{move}}$, and $n_{\text{across}}$ are the number of pick waves, the number of times a picker moved up or down, and the number of times a picker moved across the aisle, respectively.

$n_{\text{picking}}(q, (p_1, p_2, \dots, p_q))$ is the number of times the same type of object was picked from a shelf into $q$ containers with the $q$ containers getting $p_1$, $p_2$, $\dots$, $p_q$ items per container.

$n_{\text{sorting}}(q, r, ((g_{11}, g_{12}, \dots, g_{1r}), (g_{21}, g_{22}, \dots, g_{2r}), \dots, (g_{q1}, g_{q2}, \dots, g_{qr})))$ is the number of times that objects in $q$ containers were sorted into $r$ packages, with $g_{ij}$ objects being sorted from container $i$ into package $j$.

In [ ]:
from random import shuffle
from scipy.stats import zipf, poisson
from numpy.random import choice
from numpy import reshape, count_nonzero
from dataclasses import dataclass, field
from typing import List, Tuple

In [ ]:
def generate_product_probabilities_pareto(zipf_shape, products):
    # to get wikipedia zipf from scipy zipf, just truncate
    distribution = zipf(zipf_shape)
    freqs = [distribution.pmf(n+1) for n in range(len(products))]
    freqs = freqs / sum(freqs)
    shuffle(freqs)

    return dict(zip(products, freqs))

In [ ]:
# assignment of products to pick faces
def assign_products_to_pick_faces_random(products, pick_faces):
    # random
    pick_face_assignments = pick_faces.copy()
    shuffle(pick_face_assignments)

    product_to_pick_face = dict(zip(products, pick_face_assignments))
    pick_face_to_product = dict(zip(product_to_pick_face.values(), product_to_pick_face.keys()))
    
    return product_to_pick_face, pick_face_to_product

def assign_products_to_pick_faces_pareto(products, pick_faces, product_probabilities):
    # according to freq (akin to Andrew's pareto method)
    product_probs_for_assignment = list(product_probabilities.items())
    product_probs_for_assignment.sort(key=lambda x: -x[1])
    product_to_pick_face = dict(zip(map(lambda x: x[0], product_probs_for_assignment), pick_faces))
    pick_face_to_product = dict(zip(product_to_pick_face.values(), product_to_pick_face.keys()))
    
    return product_to_pick_face, pick_face_to_product

In [ ]:
@dataclass
class PickingCost:
    """
    Class to keep track of the individual actions done during
    picking, and computing the cost (at the moment time) required
    to perform them
    """

    n_fixed: int = 0
    n_move: int = 0
    n_across: int = 0
    n_picking: List[Tuple] = field(default_factory=list)
    n_sorting: List[Tuple] = field(default_factory=list)

    def __add__(self, other):
        return PickingCost(self.n_fixed + other.n_fixed,
                            self.n_move + other.n_move,
                            self.n_across + other.n_across,
                            self.n_picking + other.n_picking,
                            self.n_sorting + other.n_sorting)

In [ ]:
def sum_picking_costs(costs):
    total = costs[0]
    for cost in costs[1:]:
        total += cost
    return total

In [ ]:
# Some sample cost functions

def _cost_picking(n_picking):
    """
    We use this picking cost:
    
    * It costs 2 seconds to pick something from a shelf
    * Plus 1 second for each item
    * Plus 0.4 seconds for each extra container we have (presumably it takes a moment to figure
        out which container to place the item in)
    """
    return 3 + 1 * sum(n_picking[1:]) + 0.4 * n_picking[0]

def _cost_sorting(n_sorting):
    """
    We use this sorting cost:
    
    For each container, it costs a quadratic amount of time in the largest outgoing package
        size, except if all items are put into the same package
    """

    def _cost_sorting_per_container(container):
        # if we just move from the container to the package, there's no sorting cost
        if count_nonzero(container) <= 1:
            return 0
        
        m = max(container)
        
        return 2 + m + m*m/10

    q, r = n_sorting[:2]
    
    # turn it into a list of lists
    containers = reshape(n_sorting[2:], (q, r))

    return sum(map(_cost_sorting_per_container, containers))

def total_time(c, *, pick_cost_fn=_cost_picking, sort_cost_fn=_cost_sorting):
    # say it takes 2 minutes to print and walk around as fixed cost
    cost_fixed = 8 * c.n_fixed
    # 5 seconds to move from one shelf to another
    cost_move = 3 * c.n_move
    # 10 seconds to move across the aisle
    cost_across = 3 * c.n_across

    cost_picking = sum(map(_cost_picking, c.n_picking))
    cost_sorting = sum(map(_cost_sorting, c.n_sorting))

    return cost_fixed + cost_move + cost_across + cost_picking + cost_sorting

In [ ]:
class Product:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return "product {}".format(self.name)

In [ ]:
class PickFace:
    def __init__(self, number):
        self.number = number

    def location(self):
        """
        We'll just use some (x,y) where x is 0 or 1 (side of aisle) and y is height starting from 0
        """
        
        return (self.number % 2, self.number // 2)

    def __repr__(self):
        return "face {}".format(self.number)

In [ ]:
class Order:
    def __init__(self, product_list):
        self._products = product_list
        
    def products(self):
        return self._products
    
    def __repr__(self):
        return "Order with {} products".format(len(self._products))

In [ ]:
def generate_new_order(number_of_products, product_probabilities):
    labels, weights = zip(*product_probabilities.items())
    return Order(list(choice(labels, number_of_products, p=weights)))

In [ ]:
class PickWave:
    """
    This class represents a run through the warehouse.
    """
    
    def __init__(self, products, containers):
        # number of containers
        self.n_containers = containers
        # a dictionary that for each product contains how many times that's picked into each container
        self.product_picks = dict(zip(products, [[0] * containers for n in products]))
        # a list for each container, which tells which products are in that container
        self.products_in_containers = [[] for n in range(containers)]
        # list of packages, as (container, order) tuples
        self.packages = []
    
    def add_order(self, order, container):
        """
        Adds an order to the pick wave.
        
        Container is the container to which this order will be picked into.
        """
        if container < 0 or container >= self.n_containers:
            raise Exception("No such container")
        
        for product in order.products():
            self.product_picks[product][container] += 1
        
        self.products_in_containers[container].extend(order.products())
        
        self.packages.append((container, order))
    
    def compute_picking_cost(self, product_to_pick_face):
        # single pick face
        n_fixed = 1
        
        # compute number of up or down steps, note they come in pairs
        n_move = 2 * max([product_to_pick_face[product].location()[1] for product, picks in self.product_picks.items() if count_nonzero(picks) > 0])
        
        # move across once
        n_across = 1
        
        # this is basically the same but needs number of containers
        n_picking = [[self.n_containers] + pick for product, pick in self.product_picks.items() if count_nonzero(pick) > 0]
        
        # computer sorting operations
        q = self.n_containers
        r = len(self.packages)
        n_sorting_1 = [q, r]
        for container in range(self.n_containers):
            n_sorting_1 += [len(package[1].products()) if package[0] == container else  0 for package in self.packages]
        
        return PickingCost(n_fixed, n_move, n_across, n_picking, [n_sorting_1])

In [ ]:
class OptPickWave:
    """
    This is slightly cleverer version of the above, for optimisation, but not so nice to read, probably.
    
    Also includes other bits and pieces to make local optimisation algorithms work easier.
    """
    
    def __init__(self, products):
        # each element of this list is a container and contains a list of orders
        self.container_orders = []
        self._cost = 0
        self._cost_stale = False
        self.products = products
    
    def add_order_to_new_container(self, order):
        self._cost_stale = True
        self.container_orders.append([order])

    def add_order(self, order, container):
        self._cost_stale = True
        if container < 0 or container >= len(self.container_orders):
            raise Exception("Invalid container")
        self.container_orders[container].append(order)

    def remove_order(self, order):
        self._cost_stale = True
        self.container_orders = [[other_order for other_order in orders if other_order != order] for orders in self.container_orders]
        self.container_orders = [orders for orders in self.container_orders if len(orders) > 0]
    
    def move_order(self, order, container):
        current_container = -1
        for container, orders in enumerate(self.container_orders):
            if order in orders:
                current_container = container
                break

        if current_container == -1:
            raise Exception("Order not found")
        
        if current_container != container:
            self._cost_stale = True
            remove_order(order)
            self.container_orders[container].append(order)
    
    def compute_picking_cost(self, product_to_pick_face):
        if self._cost_stale:
            # number of containers
            self.n_containers = len(self.container_orders)
            # a dictionary that for each product contains how many times that's picked into each container
            self.product_picks = dict(zip(self.products, [[0] * self.n_containers for n in self.products]))
            # a list for each container, which tells which products are in that container
            self.products_in_containers = [[] for n in range(self.n_containers)]
            # list of packages, as (container, order) tuples
            self.packages = []

            for container, orders in enumerate(self.container_orders):
                for order in orders:
                    for product in order.products():
                        self.product_picks[product][container] += 1
                    self.products_in_containers[container].extend(order.products())
                    self.packages.append((container, order))

            # single pick face
            n_fixed = 1

            # compute number of up or down steps, note they come in pairs
            n_move = 2 * max([product_to_pick_face[product].location()[1] for product, picks in self.product_picks.items() if count_nonzero(picks) > 0])

            # move across once
            n_across = 1

            # this is basically the same but needs number of containers
            n_picking = [[self.n_containers] + pick for product, pick in self.product_picks.items() if count_nonzero(pick) > 0]

            # computer sorting operations
            q = self.n_containers
            r = len(self.packages)
            n_sorting_1 = [q, r]
            for container in range(self.n_containers):
                n_sorting_1 += [len(package[1].products()) if package[0] == container else  0 for package in self.packages]

            self._cost = PickingCost(n_fixed, n_move, n_across, n_picking, [n_sorting_1])
            self._cost_stale = False
        
        return self._cost

In [ ]:
def generate_singleton_waves_opt(batch, products):
    pick_waves = []
    
    for order in batch:
        pick_wave = OptPickWave(products)
        pick_wave.add_order_to_new_container(order)
        pick_waves.append(pick_wave)
    
    return pick_waves

In [ ]:
def generate_singleton_waves(batch, products):
    """
    This function generates pick waves where each order is just picked by itself
    """
    pick_waves = []
    
    for order in batch:
        pick_wave = PickWave(products, 1)
        pick_wave.add_order(order, 0)
        pick_waves.append(pick_wave)
    
    return pick_waves

In [ ]:
def generate_multi_order_waves(batch, products):
    """
    Pick each order into its own container
    """
    pick_waves = []
    
    pick_wave = PickWave(products, len(batch))

    for container, order in enumerate(batch):
        pick_wave.add_order(order, container)
    
    pick_waves.append(pick_wave)
    
    return pick_waves

In [ ]:
def generate_batch_order_waves(batch, products):
    """
    Pick each order into its own container
    """
    pick_waves = []
    
    pick_wave = PickWave(products, 1)

    for order in batch:
        pick_wave.add_order(order, 0)
    
    pick_waves.append(pick_wave)
    
    return pick_waves

In [ ]:
def generate_multi_batch_order_waves(batch, containers, products):
    """
    Pick each order into its own container
    """
    pick_waves = []
    
    pick_wave = PickWave(products, containers)

    for container, order in enumerate(batch):
        pick_wave.add_order(order, container % containers)
    
    pick_waves.append(pick_wave)
    
    return pick_waves

In [ ]:
# number of pick faces
no_pick_faces = 100

# zipf's law shape parameter
zipf_shape = 1.07

In [ ]:
# the pick face labels
pick_faces = [PickFace(n) for n in range(no_pick_faces)]

# there's the same number of products as pick faces
products = [Product(n) for n in range(no_pick_faces)]

product_probabilities = generate_product_probabilities_pareto(zipf_shape, products)
product_to_pick_face, pick_face_to_product = assign_products_to_pick_faces_pareto(products, pick_faces, product_probabilities)

pick_faces, products, product_probabilities, product_to_pick_face, pick_face_to_product

In [ ]:
# Generate a batch of orders
batch = [generate_new_order(poisson(25).rvs(), product_probabilities) for n in range(60)]

In [ ]:
# Picking orders individually
singleton_waves = generate_singleton_waves_opt(batch, products)
total = PickingCost()
for wave in singleton_waves:
    total += wave.compute_picking_cost(product_to_pick_face)
total_time(total)

In [ ]:
# Picking orders individually
singleton_waves = generate_singleton_waves(batch, products)
total = PickingCost()
for wave in singleton_waves:
    total += wave.compute_picking_cost(product_to_pick_face)
total_time(total)

In [ ]:
# Picking all orders at the same time
multi_order_waves = generate_multi_order_waves(batch, products)
total = PickingCost()
for wave in multi_order_waves:
    total += wave.compute_picking_cost(product_to_pick_face)
total_time(total)

In [ ]:
# Picking all orders at the same time but sort at end
multi_order_waves = generate_batch_order_waves(batch, products)
total = PickingCost()
for wave in multi_order_waves:
    total += wave.compute_picking_cost(product_to_pick_face)
total_time(total)

In [ ]:
# Picking all orders at the same time into at most 5 containers but sort at end
multi_order_waves = generate_multi_batch_order_waves(batch, 5, products)
total = PickingCost()
for wave in multi_order_waves:
    total += wave.compute_picking_cost(product_to_pick_face)
total_time(total)